In [72]:
import os
import time
import datetime
import re
import shutil
from collections import deque
import argparse

import numpy as np
import torch
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

from simulation.simulations.data_generator import DataGenerator
from TransformerMOT.util.misc import save_checkpoint, update_logs
from TransformerMOT.util.load_config_files import load_yaml_into_dotdict
from TransformerMOT.util.plotting import output_truth_plot, compute_avg_certainty, get_constrastive_ax, get_false_ax, \
    get_total_loss_ax, get_state_uncertainties_ax
from TransformerMOT.util.logger import Logger
from TransformerMOT.models.BOMTv1 import BOMT
from simulation.simulations.data_generator import DataGenerator, get_single_training_example


In [73]:
task_params = r"C:\Users\chiny\OneDrive - Nanyang Technological University\Y3S2 (Internship)\MultiTracking\configs\tasks\task2.yaml"
model_params = r"C:\Users\chiny\OneDrive - Nanyang Technological University\Y3S2 (Internship)\MultiTracking\configs\models\BOMTv1.yaml"

params = load_yaml_into_dotdict(task_params)
params.update(load_yaml_into_dotdict(model_params))

if params.general.pytorch_and_numpy_seed is None:
    random_data = os.urandom(4)
    params.general.pytorch_and_numpy_seed = int.from_bytes(random_data, byteorder="big")
print(f'Using seed: {params.general.pytorch_and_numpy_seed}')

if params.training.device == 'auto':
    params.training.device = 'cuda' if torch.cuda.is_available() else 'cpu'

Using seed: 4121252701


In [91]:
from TransformerMOT.util.misc import NestedTensor

data_generator = DataGenerator(params=params)
training_nested_tensor, labels, unique_measurement_ids, target_coordinates = data_generator.get_batch()

zeros_tensors = torch.zeros_like(training_nested_tensor.tensors)
zeros_tensors[:,:,-1] = training_nested_tensor.tensors[:,:,-1]
test_nested_tensor = NestedTensor(zeros_tensors, training_nested_tensor.mask).to("cuda")


In [92]:
model1 = BOMT(params).to("cuda")
checkpoint = torch.load(r"results\train2_2phases_1\checkpoints\checkpoint_gradient_step_1615.pth",map_location="cuda")
model1.load_state_dict(checkpoint["model_state_dict"])
res = model1.forward_phase1(training_nested_tensor.to("cuda"))

In [77]:
torch.set_printoptions(threshold=10_000)

In [93]:
res

tensor([[[25.8384, 26.2129],
         [25.2174, 24.2178],
         [24.3761, 24.0273],
         [26.9599, 26.6365],
         [28.0212, 27.2757],
         [24.1540, 24.6152],
         [24.1794, 25.2258],
         [24.9913, 25.1236],
         [25.3043, 24.5982],
         [28.6770, 27.7105],
         [23.6769, 24.0265],
         [24.5191, 24.7275],
         [24.6801, 24.8771],
         [24.4102, 24.7684],
         [24.3800, 24.2282],
         [24.0156, 25.3825],
         [25.5654, 24.8974],
         [27.2605, 26.2426],
         [25.1341, 24.9868],
         [27.1133, 27.7697],
         [25.5197, 25.0152],
         [24.5508, 24.1267],
         [26.3138, 26.1577],
         [24.0772, 25.1699],
         [23.1182, 22.1819],
         [25.0884, 25.9142],
         [25.0694, 25.8521],
         [23.0899, 23.8142],
         [21.6098, 21.3745],
         [23.6369, 23.3299],
         [24.6227, 24.3571],
         [28.2180, 28.0943],
         [24.1358, 24.1842],
         [24.7384, 23.3480],
         [24.9

In [88]:
error = res - torch.zeros_like(target_coordinates)

distance = (error[:,:,0]**2 + error[:,:,1]**2)**0.5
distance[0].mean()

tensor(22.7238, device='cuda:0', grad_fn=<MeanBackward0>)

In [79]:
model1.rf_layer.d_embeddings.weight

Parameter containing:
tensor([[ 0.1621],
        [ 0.2711],
        [-0.2404],
        [-0.3095],
        [ 0.0786],
        [-0.0709],
        [-0.2373],
        [ 0.1465],
        [-0.0226],
        [ 0.1357],
        [ 0.2016],
        [-0.2708],
        [ 0.3077],
        [-0.0277],
        [ 0.1415],
        [ 0.1554],
        [ 0.0398],
        [-0.2531],
        [-0.1453],
        [ 0.0678],
        [-0.1604],
        [-0.2250],
        [-0.0318],
        [-0.1228],
        [ 0.1288],
        [ 0.2426],
        [ 0.0851],
        [ 0.0351],
        [-0.1671],
        [-0.0023],
        [-0.1245],
        [ 0.2097],
        [ 0.1029],
        [ 0.3047],
        [-0.2689],
        [-0.1310],
        [-0.2442],
        [-0.2059],
        [-0.1632],
        [-0.2090],
        [-0.1565],
        [-0.1932],
        [ 0.0299],
        [ 0.1527],
        [-0.1464],
        [-0.2658],
        [-0.1321],
        [ 0.0265],
        [ 0.1879],
        [ 0.2459],
        [-0.2817],
        [

In [80]:
model1 = BOMT(params).to("cuda")
checkpoint = torch.load(r"results\train2_new\checkpoints\checkpoint_gradient_step_2005.pth",map_location="cuda")
model1.load_state_dict(checkpoint["model_state_dict"])
res = model1(training_nested_tensor.to("cuda"))
res_1 = model1(test_nested_tensor.to("cuda"))

In [81]:
res[0].positions

tensor([[[49.9703, 51.4373],
         [50.0642, 51.5402],
         [49.5743, 51.0242],
         [49.9042, 51.3566],
         [49.6967, 51.1336],
         [50.0122, 51.4648],
         [50.1581, 51.6077],
         [48.6753, 50.0724],
         [50.5440, 52.0342],
         [49.9623, 51.4195],
         [49.4918, 50.9411],
         [49.6094, 51.0329],
         [49.3637, 50.7891],
         [49.4393, 50.9053],
         [50.6885, 52.1840],
         [49.6882, 51.1289]],

        [[50.0809, 51.5525],
         [50.3694, 51.8646],
         [50.0317, 51.4959],
         [50.1660, 51.6361],
         [49.4020, 50.8300],
         [49.9450, 51.4014],
         [49.9248, 51.3999],
         [50.2248, 51.6804],
         [50.1477, 51.6183],
         [49.9967, 51.4718],
         [49.7725, 51.2553],
         [49.6595, 51.1107],
         [49.9746, 51.4431],
         [50.6673, 52.1623],
         [50.2902, 51.7656],
         [50.6609, 52.1647]],

        [[49.5471, 50.9937],
         [49.3157, 50.7400],
         [

In [82]:
model1.rf_layer.d_embeddings.weight

Parameter containing:
tensor([[-0.2513],
        [-0.1306],
        [ 0.0383],
        [ 0.2029],
        [ 0.0343],
        [ 0.0881],
        [-0.1261],
        [-0.1580],
        [-0.2722],
        [ 0.0630],
        [-0.1347],
        [ 0.1349],
        [ 0.0363],
        [-0.1939],
        [-0.0149],
        [-0.0254],
        [-0.0559],
        [-0.2344],
        [ 0.0210],
        [ 0.2574],
        [ 0.2565],
        [-0.2365],
        [-0.1967],
        [-0.1749],
        [ 0.2456],
        [-0.1519],
        [-0.0932],
        [ 0.2648],
        [-0.2740],
        [-0.1737],
        [ 0.2630],
        [ 0.2208],
        [-0.0836],
        [ 0.0826],
        [-0.0648],
        [-0.2234],
        [-0.2156],
        [ 0.2931],
        [-0.1939],
        [-0.2458],
        [-0.2658],
        [ 0.2779],
        [-0.2080],
        [-0.0807],
        [-0.0223],
        [ 0.1780],
        [ 0.2675],
        [-0.2822],
        [ 0.0521],
        [ 0.2838],
        [-0.3008],
        [

In [83]:
res[-1]

{'intermediate_attention': []}